In [1]:
from Task1 import Data

data_info = {
    "other_data": "./data-2/other.xlsx",
    "other_folder": "./data-2/other/*",
    "apple_data": "./data-2/apple.xlsx",
    "apple_file": "./data-2/apple/Apple_Full text.XML",
}

apple_df = Data(**data_info).apple()
other_df = Data(**data_info).other()

len(apple_df), len(other_df)

apple company file exist.
other company file exist.


c:\Users\cglog\Desktop\project-nlp-patent-similarity\Task1.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Applicant_verify'] = data['Applicant'].apply(lambda x: x.capitalize())
c:\Users\cglog\Desktop\project-nlp-patent-similarity\Task1.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Applicant_verify'] = data['Applicant_verify'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]", "", x))
c:\Users\cglog\Desktop\project-nlp-patent-similarity\Task1.py:49: SettingWithCopyWarning: 
A value is trying to

(352, 60729)

In [ ]:
apple_df.info()

In [2]:
other_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60729 entries, 0 to 65277
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Country Code                  60729 non-null  object
 1   Granted Number                60729 non-null  object
 2   Examined Pub. Number          60729 non-null  object
 3   Unexamined Pub. Number        60729 non-null  object
 4   Kind Code                     60729 non-null  object
 5   Application Number            60729 non-null  object
 6   Application Date              60729 non-null  object
 7   Unexamined Pub. Date          60729 non-null  object
 8   Examined Pub. Date            60729 non-null  object
 9   Status[US,JP,KR,CN,EP,CA,AU]  60729 non-null  object
 10  Applicant                     60729 non-null  object
 11  Orig. IPC(All)                60729 non-null  object
 12  Curr. IPC(All)                60729 non-null  object
 13  Title           

# 公司歸屬

In [10]:
from tqdm import tqdm
Applicant_set  = other_df['Applicant_verify'].unique()
associate_code = {}
for i in tqdm(other_df['Applicant_verify']):
    for i2 in Applicant_set:
        if i in i2:
            if associate_code.get(i) is None:
                associate_code[i] = [i2]
            else:
                associate_code[i].extend([i2])

100%|██████████| 60729/60729 [02:20<00:00, 432.87it/s]


In [15]:
for k in associate_code.keys():
    associate_code[k] = list(set(associate_code[k]))

In [18]:
associate_code
new_applicant_code = {index: k for index, k in enumerate(associate_code.keys())}

In [21]:
for applicant_code, applicant_name in new_applicant_code.items():
    new_applicant_code[applicant_code] = [applicant_name]
    new_applicant_code[applicant_code].extend(associate_code[applicant_name])

In [23]:
def give_new_applicant_id(applicant_verify):
    for code, name in new_applicant_code.items():
        if applicant_verify in name:
            return code
    return -1

other_df['new_applicant_id'] = other_df['Applicant_verify'].map(give_new_applicant_id)

In [42]:
other_df_process_company_count = other_df.groupby(by=['new_applicant_id', 'Applicant']).count().reset_index().loc[:,["new_applicant_id", "Applicant", "Country Code"]].rename({"Country Code": "count"}, axis=1)
other_df_process_company_count = other_df_process_company_count.sort_values(by=['new_applicant_id', 'count'])
other_df_process_company_count.to_excel("公司專利數量(特殊處理).xlsx")

In [49]:
other_df.drop(labels=['Applicant Id', 'Applicant_verify'], axis=1).to_excel("./公司專利(公司名已處理).xlsx")

In [19]:
## 驗證
import re
def company_v(company:str):
    company = re.sub(r"[^a-zA-Z0-9]", "", company)
    company = company.capitalize()
    return company

In [20]:
other_df['company_verify'] = other_df['Applicant'].map(company_v)

In [21]:
counts_v = other_df.value_counts(subset='company_verify').reset_index().rename({0:'專利數量'},axis=1)
counts_v

,company_verify,專利數量
0,Koninklijkephilipsnv,1490
1,Samsungelectronicscoltd,934
2,Medtronicinc,874
3,Abbottdiabetescareinc,811
4,Dexcominc,754
...,...,...
15221,Interuniversityresearchinstitutecorporationnih...,1
15222,Intervalvemedicalinc,1
15223,Intesectionmedicalinc,1
15224,Intincoltd,1


In [ ]:
counts_v

In [18]:
counts_v[counts_v['company_verify'].str.contains("philips")]

,company_verify,專利數量
0,Koninklijke philips nv,1486
29,Koninklijke philips electronics nv,207
413,Koninklijke philips electronics n v,18
1931,Koninklijke philips n v,4
1938,Koninklijke philips nv wisconsin alumni resea...,4
2614,Koninklijke philips electronic nv,3
3820,Koninklijke philips nv boston scientific scim...,2
3821,Koninklijke philips electronics,2
3823,Koninklijke philips electroncs nv,2
3827,Koninklije philips nv,2


In [ ]:
"MICCK".capitalize()

In [ ]:
import pandas as pd

df2 = pd.read_excel("./data-2/other/00001-10000.xlsx")
df3 = pd.read_excel("./data-2/other/10001-20000.xlsx")
df4 = pd.read_excel("./data-2/other/20001-30000.xlsx")
df5 = pd.read_excel("./data-2/other/30001-40000.xlsx")
df6 = pd.read_excel("./data-2/other/30001-40000.xlsx")
df7 = pd.read_excel("./data-2/other/40001-50000.xlsx")
df8 = pd.read_excel("./data-2/other/50001-60000.xlsx")
D

In [ ]:
df9.value_counts(subset='Applicant').reset_index().rename({0:'專利數量'},axis=1)